In [27]:
! pip install openpyxl datetime pandas

In [28]:
import os
from openpyxl import Workbook
from datetime import datetime
import pandas as pd

In [29]:
user_input = '/Users/atharvchandratre/Documents/prism_n3_le20_d202/prism_n3_le20_d202_3.log' # Enter path to log here
assert os.path.exists(user_input), "I did not find the file at "+str(user_input)
f = open(user_input,'r')
print("Log File found, beginning analysis...")
f.close()

Log File found, beginning analysis...


In [30]:
ProposerBlockMined = 'ProposerBlockMined'
ValidProposerBlockDownloaded = 'ValidProposerBlockDownloaded'
ProposerBlockConfirmed = 'ProposerBlockConfirmed'
ProposerBlockRolledBack = 'ProposerBlockRolledBack'
epoch = datetime.utcfromtimestamp(0)

hashToNumber = {}
hashToCreationTimestamp = {}
hashToCreationEvent = {}
hashToConfirmationTimestamp = {}
hashToRollbackTimestamp = {}

In [31]:
def getTimestamp(timestr):
    dateTimeObj = datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S,%f')
    delta = dateTimeObj - epoch
    return delta.total_seconds()*1000

In [32]:
with open(user_input, 'r') as f:
    for line in f:
        if "BENCHMARKING:" not in line:
            continue
        benchmarkSubstrings = line[line.find("BENCHMARKING"):-1].split(':')
        dateTimeString = line.split('[')[0][:-1]
        timestamp = getTimestamp(dateTimeString)
        event = benchmarkSubstrings[1]
        blockNumber = benchmarkSubstrings[3]
        blockHash = benchmarkSubstrings[-1][1:-1]
        if event == ProposerBlockMined or event == ValidProposerBlockDownloaded:
            if hashToCreationTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToCreationEvent[blockHash] = event
            hashToCreationTimestamp[blockHash] = timestamp
        if event == ProposerBlockConfirmed:
            if hashToConfirmationTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToConfirmationTimestamp[blockHash] = timestamp
        if event == ProposerBlockRolledBack:
            if hashToRollbackTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToRollbackTimestamp[blockHash] = timestamp

f.close()

In [33]:
df = pd.DataFrame(columns=['Block Hash','Block Number','Timestamp - Creation','Event','Timestamp - Confirmation','Timestamp - Rollback'])

for index,key in enumerate(hashToNumber.keys()):
    df.loc[index] = [key,
        int(hashToNumber.get(key,'')),
        hashToCreationTimestamp.get(key,''),
        hashToCreationEvent.get(key,''),
        hashToConfirmationTimestamp.get(key,''),
        hashToRollbackTimestamp.get(key,'')]

df.sort_values('Block Number')

,Block Hash,Block Number,Timestamp - Creation,Event,Timestamp - Confirmation,Timestamp - Rollback
0,0x30d9b6946f0168b15a60dd5463364864fa678ccc3e27...,1,1.639965e+12,ValidProposerBlockDownloaded,1639968431315.0,
1,0xd448e65412657030115d0645057db1504b59065b338e...,2,1.639965e+12,ValidProposerBlockDownloaded,1639968725644.0,
2,0x9801770612424900b503f726171d03e223b28599bcb3...,3,1.639965e+12,ValidProposerBlockDownloaded,1639968726590.0,
3,0xa090c8da22dd0866678e3ee7e45f5878aca5ae3940cf...,4,1.639965e+12,ValidProposerBlockDownloaded,1639968727298.0,
4,0xc0a12ad9e808dd53a47740802f3ba15ac8a62ade6bf1...,5,1.639965e+12,ValidProposerBlockDownloaded,1639968728137.0,
...,...,...,...,...,...,...
118,0xf7eb851f52e6c48bfddcee4ba4252c13c74eedf16194...,87,1.639971e+12,ProposerBlockMined,,
119,0xd4f8101e9204c11411aac8112089bcd7a6d55389e036...,88,1.639971e+12,ProposerBlockMined,,
120,0x36941d44f348939bda5281539b26cbfe4abb7a0cdf2e...,89,1.639971e+12,ProposerBlockMined,,
121,0x15e1947e90e02e8f60d21bdf04c504034ab868bb710d...,90,1.639971e+12,ProposerBlockMined,,


In [34]:
df['Latency'] = df[df['Timestamp - Confirmation']!='']['Timestamp - Confirmation']-df[df['Timestamp - Confirmation']!='']['Timestamp - Creation']
df['Mining Time'] = df['Timestamp - Creation'].diff()
df['Confirmation Depth']=df['Latency']/df['Mining Time']

In [35]:
print("Mean Latency =",df['Latency'].dropna().mean())
print("Median Latency =",df['Latency'].dropna().median())
print("Mean Mining Time =",df['Mining Time'].dropna().mean())
print("Median Mining Time =",df['Mining Time'].dropna().median())
print("Mean Confirmation Depth =",df['Confirmation Depth'].dropna().mean())
print("Median Confirmation Depth =",df['Confirmation Depth'].dropna().median())

Mean Latency = 3498213.882352941
Median Latency = 3455308.0
Mean Mining Time = 53457.11475409836
Median Mining Time = 50164.5
Mean Confirmation Depth = 123.05236877850881
Median Confirmation Depth = 62.92287072934435


In [36]:
creationTimestamps = df['Timestamp - Creation']
confirmationTimestamps = df[df['Timestamp - Confirmation']!='']['Timestamp - Confirmation']
insertionPoints = pd.DataFrame(creationTimestamps.searchsorted(value=confirmationTimestamps),columns=['Insertion Points'])
cdArray = insertionPoints['Insertion Points'] - insertionPoints['Insertion Points'].index
print("Trial Mean Confirmation Depth =",cdArray.mean())
print("Trial Median Confirmation Depth =",cdArray.median())

Trial Mean Confirmation Depth = 74.37254901960785
Trial Median Confirmation Depth = 74.0
